# Decrypting data files

This file is part of the **"Running Virtual Reality Experiments Online: A Brief Introduction and Tutorial"** and the accompanying **onlineVR-toolbox**. 

**Authors:** Levi Kumle, Alfie Brazier, Joel Kovoor, Johannes Keil, Anna C. Nobre & Dejan Draschkow

**last edited:** 08/05/2025

---
---
for further details, see: 

- [onlineVR-toolbox](https://github.com/lkumle/onlineVR-toolbox)  

- [Step-by-step Notebooks](https://lkumle.github.io/onlineVRtoolbox_tutorials/)

- [Tutorial paper]()


---
---


### What is this script doing?
This script handles the decryption of data recieved from the unity template task using AES-CBC. 

1.  **Set up:** configure script by specifying details needed for decrytion 
2.  **Load decryption fucntion:** The decrypt_aes function decrypts data that was encrypted using the AES algorithm in CBC mode, with padding applied using the PKCS7 scheme.
3.  **Apply decryption to all file:** load in file, decrypt, decode from bytes to text, and save as csv

---
---

### How to use this script:

1. Add the **encrytion key** used to encrypt data within the VR task
2. Specify file paths: 
    - **file_path_in:** location of encrypted files
    - **file_path_out:** where should decrypted files be stored?
  
  
3. Run the rest of the script. 
    

In [2]:
# load libraries
import os
import struct # for bytewise operations
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes 
from cryptography.hazmat.primitives import padding

---
---
## 1. Set up:  encryption key and file paths

In [13]:
# SET UP ------------------------------------------ #
# Define decryption key, IV, and file paths
encryption_key = b"tutorial-example"  # Must be 16, 24, or 32 bytes


# where are encrypted data files saved?
file_path_in = "..."
# where to store decrypted data files?
file_path_out = "..."

---
---
##  2. Define decryption function

Simply run this cell. 

In [10]:
# Function to decrypt AES-CBC encrypted data
def decrypt_aes(encrypted_data, key, iv_length = 16):
    
    """
    Decrypts AES-CBC encrypted data and removes the padding using PKCS7.

    This function decrypts the provided encrypted data using the AES algorithm
    in CBC (Cipher Block Chaining) mode, then removes the padding that was 
    added during encryption using the PKCS7 padding scheme.

    Parameters:
    - encrypted_data (bytes): The encrypted data to be decrypted.
    - key (bytes): The AES encryption key (must be 16 bytes for AES-128).
    - iv_lengths (int): Lengths of random AES initialization vector (IV). 

    Returns:
    - bytes: The decrypted data with padding removed, i.e., the original plaintext.
    
    """
    
    # Extract the IV from the beginning of the encrypted data
    iv = encrypted_data[:iv_length]

    # Remove the IV from the encrypted data
    data = encrypted_data[iv_length:]
    
    # Initialize the cipher for decryption using AES-CBC mode with provided key and IV
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    decryptor = cipher.decryptor()
    
    # Decrypt the data
    decrypted_padded = decryptor.update(data) + decryptor.finalize()
    
    # Remove padding using PKCS7
    unpadder = padding.PKCS7(128).unpadder() #128 referrs to block size used by AES encryption
    decrypted_data = unpadder.update(decrypted_padded) + unpadder.finalize()
    
    return decrypted_data 

---
---
## 3. Load, decrypt, and save data

The code below goes through al files found in file_path_in, loads and decrypts the file, before saving it to file_path_out as a .csv


In [ ]:
# list all files in file_path_in
files = os.listdir(file_path_in)

# go through all files
for file in files:
    
    print("Decrypting:  "  + file) # quick update
    
    # specify path to file
    file_path_data = f"{file_path_in}/{file}"

    # 1. Load encrypted data
    with open(file_path_data, "rb") as f:
        encrypted_data = f.read()

    # 2. decrypt (returns decrypted data as binary)
    decrypted_data = decrypt_aes(encrypted_data,encryption_key, )


    # 3. Convert to text (from bytes)
    decrypted_text = decrypted_data.decode("utf-8")


    # 4. save as csv
    file_name_out = file.replace(".bytes", ".csv") 
    file_path_data_out = file_path_out + file_name_out  
    
    with open(file_path_data_out, "w", encoding="utf-8") as f:
        f.write(decrypted_text)
    